In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/all_dataset/AWF_data.zip'
from google.colab import files
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
import shutil
import os
directory = "/content/dataset"
if any(os.listdir(directory)):
  print('dataset exists')
  !unzip /content/dataset/AWF_data.zip -d /content/dataset
else:
  !unzip /content/dataset/AWF_data.zip -d /content/dataset

dataset exists
Archive:  /content/dataset/AWF_data.zip
  inflating: /content/dataset/dataset/X_test.npy  
  inflating: /content/dataset/dataset/X_train.npy  
  inflating: /content/dataset/dataset/X_valid.npy  
  inflating: /content/dataset/dataset/y_test.npy  
  inflating: /content/dataset/dataset/y_train.npy  
  inflating: /content/dataset/dataset/y_valid.npy  


In [5]:
!gdown 15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD


Downloading...
From (original): https://drive.google.com/uc?id=15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD
From (redirected): https://drive.google.com/uc?id=15D1vov_iXPWMpGO6-xnb-M6aDXWPxtHD&confirm=t&uuid=e249a5ee-50e1-45db-b956-0bfd93921f3b
To: /content/X_test_Unmon_NoDef.pkl
100% 801M/801M [00:05<00:00, 147MB/s]


In [6]:
mv /content/X_test_Unmon_NoDef.pkl /content/dataset/openset.pkl

In [8]:
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from utility import LoadDataNoDefCW
from Model_NoDef import DFNet
import random
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd

from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [9]:
def representative_data_gen_Open():
  for input_value in tf.data.Dataset.from_tensor_slices(X__open).batch(1).take(1000):
    # Model has only one input so each data point has one element.
    yield [input_value]

def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3))
    else:
        return str(size)


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

In [10]:
import gc
gc.collect()

18

In [11]:
NB_EPOCH = 10   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 1500 # Packet sequence length
OPTIMIZER = Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08) # Optimizer

NB_CLASSES = 200 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

random.seed(0)

print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_valid, y_valid, X_test, y_test = LoadDataNoDefCW()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')

# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]

# Convert class vectors to categorical classes matrices
y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

Number of Epoch:  10
Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario
Data dimensions:
X: Training data's shape :  (100000, 1500)
y: Training data's shape :  (100000,)
X: Validation data's shape :  (40000, 1500)
y: Validation data's shape :  (40000,)
X: Testing data's shape :  (40000, 1500)
y: Testing data's shape :  (40000,)


In [12]:
model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

filepath = '/content/trained_models/AWF.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Start training
history = model.fit(X_train, y_train,
		batch_size=BATCH_SIZE, epochs=NB_EPOCH,
		verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

#Start evaluating model with testing data

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/DF_without_softmax.hdf5')
model_penultimate.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

score_test = model_penultimate.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])

del model


Model compiled
Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.70505, saving model to /content/trained_models/AWF.hdf5
782/782 - 56s - loss: 2.9241 - accuracy: 0.3227 - val_loss: 1.2462 - val_accuracy: 0.7050 - 56s/epoch - 72ms/step
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_accuracy improved from 0.70505 to 0.84525, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 1.3315 - accuracy: 0.6554 - val_loss: 0.6057 - val_accuracy: 0.8453 - 40s/epoch - 51ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.84525 to 0.89935, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.8506 - accuracy: 0.7765 - val_loss: 0.3938 - val_accuracy: 0.8993 - 40s/epoch - 52ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.89935 to 0.91310, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.6250 - accuracy: 0.8367 - val_loss: 0.3380 - val_accuracy: 0.9131 - 40s/epoch - 51ms/step
Epoch 5/10

Epoch 5: val_accuracy improved from 0.91310 to 0.94275, saving model to /content/trained_models/AWF.hdf5
782/782 - 40s - loss: 0.4834 - accuracy: 0.8750 - val_loss: 0.2367 - val_accuracy: 0.9427 - 40s/epoch - 51ms/step
Epoch 6/10

Epoch 6: val_accuracy improved from 0.94275 to 0.95325, saving model to /co

1250/1250 - 7s - loss: 16.0302 - accuracy: 0.9694 - 7s/epoch - 5ms/step
Testing closed accuracy_without_norm: 0.9693750143051147


In [16]:
import _pickle as cPickle
dataset_dir = "/content/dataset/"
with open( dataset_dir+"openset.pkl","rb") as f:
        X_open = cPickle.load(f,encoding='latin1')
X_open = np.array(X_open)
X_open=X_open[:,:1500]
print(X_open.shape)
X_open = X_open.astype('float32')
X_open = X_open[:,:,np.newaxis]
y_open = np.array([NB_CLASSES]*len(X_open))
y_open = y_open.astype('uint8')

(20000, 1500)


In [13]:
def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]


y_train=np.argmax(y_train, axis=1)
y_valid=np.argmax(y_valid, axis=1)
y_test=np.argmax(y_test, axis=1)

tflite_model_predictions_test=model_penultimate.predict(X_test)

acc_ = accuracy_score(np.argmax(tflite_model_predictions_test,axis=1), y_test[:len(tflite_model_predictions_test)])
print('Test accuracy TFLITE model :', acc_)

del X_test
gc.collect()

1250/1250 [==============================] - 6s 4ms/step
Test accuracy TFLITE model : 0.969375


2903

In [15]:

for i in range(NB_CLASSES):
  variable_name = f"Mean_{i}"
  locals()[variable_name]=np.array([0] * NB_CLASSES)

tflite_model_predictions=model_penultimate.predict(X_train)

del X_train

count=[0]*NB_CLASSES
txt_O = "Mean_{Class1:.0f}"
Means={}
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=np.array([0]*NB_CLASSES)

for i in range(len(tflite_model_predictions)):
  k=np.argmax(tflite_model_predictions[i])
  if (np.argmax(tflite_model_predictions[i])==y_train[i]):
    Means[txt_O.format(Class1=y_train[i])]=Means[txt_O.format(Class1=y_train[i])]+tflite_model_predictions[i]
    count[y_train[i]]+=1
print("Counts: ",count)

Mean_Vectors=[]
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]
  Mean_Vectors.append(Means[txt_O.format(Class1=i)])

Mean_vectors=np.array(Mean_Vectors)


Mean_vectors = np.array(Mean_Vectors)
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)

tflite_model_predictions=model_penultimate.predict(X_valid)

del X_valid
gc.collect()

3125/3125 [==============================] - 17s 5ms/step
Counts:  [487, 459, 497, 493, 495, 483, 494, 497, 495, 492, 474, 495, 494, 498, 497, 495, 493, 490, 493, 491, 500, 492, 496, 492, 493, 479, 488, 497, 489, 498, 493, 492, 494, 476, 485, 489, 489, 487, 492, 349, 495, 497, 494, 493, 497, 496, 485, 497, 469, 473, 498, 490, 488, 490, 488, 495, 499, 490, 496, 482, 489, 490, 496, 498, 494, 489, 492, 497, 494, 492, 492, 498, 474, 494, 489, 493, 499, 481, 490, 490, 493, 493, 471, 493, 474, 498, 483, 496, 474, 493, 496, 494, 482, 496, 498, 486, 491, 483, 496, 491, 500, 500, 489, 493, 493, 497, 499, 489, 495, 495, 497, 489, 495, 498, 493, 496, 495, 486, 480, 495, 494, 496, 465, 490, 495, 494, 496, 479, 494, 492, 496, 486, 494, 499, 495, 493, 497, 494, 499, 497, 489, 490, 495, 497, 490, 495, 499, 483, 452, 494, 497, 500, 497, 497, 484, 495, 494, 498, 483, 498, 497, 483, 492, 493, 494, 491, 496, 491, 485, 499, 494, 497, 488, 498, 497, 494, 485, 496, 495, 497, 498, 497, 497, 494, 499, 479, 49

1226

In [17]:
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]


Indexes=[]
for i in range(NB_CLASSES):
  Indexes.append([])

Values={}
for i in range(NB_CLASSES):
  Values[i]=[0]*NB_CLASSES


for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        Values[y_valid[i]][k]+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist

for i in range(NB_CLASSES):
  Tot=0
  for l in range(50):
      Min=min(Values[i])
      Tot+=Min
      Indexes[i].append(Values[i].index(Min))
      Values[i][Values[i].index(Min)]=1000000

Indexes=np.array(Indexes)

np.save('/content/temp_variables/Values.npy',np.array(Values))
np.save('/content/temp_variables/Indexes.npy',Indexes)

print()
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Distances[txt_1.format(Class1=y_valid[i])].append(dist)

TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.9)]



Threasholds_1=np.array(TH)
np.save('/content/temp_variables/Threasholds_1.npy',Threasholds_1)
print("Thresholds for method 1 calculated")
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=(np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist)
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.1)]



Threasholds_2=np.array(TH)
np.save('/content/temp_variables/Threasholds_2.npy',Threasholds_2)


print("Thresholds for method 2 calculated")

##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]) and k in Indexes[y_valid[i]]:
        Tot+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])
    Tot=dist/Tot
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.9)]

Threasholds_3=np.array(TH)
np.save('/content/temp_variables/Threasholds_3.npy',Threasholds_3)
print("Thresholds for method 3 calculated")



Thresholds for method 1 calculated

Thresholds for method 2 calculated

Thresholds for method 3 calculated


In [18]:
############################################################################################

######################## Evaluating results ##########################
tflite_model_predictions_open=model_penultimate.predict(X_open)

del X_open
gc.collect()

625/625 [==============================] - 3s 5ms/step


612

In [19]:
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):

    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_test[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):

    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i]-Mean_vectors[d])>Threasholds_1[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

y_test=y_test[:len(prediction_classes)]
y_open=y_open[:len(prediction_classes_open)]

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
print("Micro")

F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)




#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])-dist

    if Tot<Threasholds_2[d]:
      prediction_classes_open.append(NB_CLASSES)
    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)



#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes=[]
for i in range(len(tflite_model_predictions_test)):
    d=np.argmax(tflite_model_predictions_test[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_test[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=d:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_test[i])

    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes.append(NB_CLASSES)

    else:
      prediction_classes.append(d)

prediction_classes=np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test[:len(prediction_classes)])
print('Test accuracy Normal model_Closed_set :', acc_Close)


prediction_classes_open=[]
for i in range(len(tflite_model_predictions_open)):
    d=np.argmax(tflite_model_predictions_open[i], axis=0)
    dist=np.linalg.norm(Mean_vectors[d]-tflite_model_predictions_open[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(d) and k in Indexes[d]:
        Tot+=np.linalg.norm(Mean_vectors[k]-tflite_model_predictions_open[i])
    Tot=dist/Tot
    if Tot>Threasholds_3[d]:
      prediction_classes_open.append(NB_CLASSES)

    else:
      prediction_classes_open.append(d)

prediction_classes_open=np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open[:len(prediction_classes_open)])
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix=[]
for i in range(NB_CLASSES+1):
  Matrix.append(np.zeros(NB_CLASSES+1))


for i in range(len(y_test)):
  Matrix[y_test[i]][prediction_classes[i]]+=1

for i in range(len(y_open)):
  Matrix[y_open[i]][prediction_classes_open[i]]+=1

F1_Score=New_F1_Score(Matrix)

print("Average Novel F1_Score: ", F1_Score)

print()
print("Micro")
F1_Score=Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
print("Macro")
F1_Score=Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)


############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.87725
Test accuracy Normal model_Open_set : 0.9515
Precision:  0.9758458968038557
Recall:  0.8840715705765407
Average novel F1_Score:  0.9276945080844701

Micro
Micro Precision:  0.9637726935647779
Micro Recall:  0.87725
Average Micro F1_Score:  0.9184781847530897

Macro
Precision: 0.9684587513904142
Recall: 0.87725
Average Macro F1_Score:  0.9206007550370328

############## Distance Method 2 #################################

Test accuracy Normal model_Closed_set : 0.9688
Test accuracy Normal model_Open_set : 0.9725
Precision:  0.96567281089131
Recall:  0.9722994880546075
Average Novel F1_Score:  0.9689748148863533

Micro
Micro Precision:  0.9579275226182825
Micro Recall:  0.9688
Average Micro F1_Score:  0.9633330797442367

Macro
Precision: 0.960575141405872
Recall: 0.9688
Average Macro F1_Score:  0.9646700345610936

############## Distance Method 3 ##########################